In [1]:
import numpy as np

def beale_function(x, y):
    #f(x_1,x_2) = (1.5 - x_1 + x_1 x_2)^2 + (2.25 - x_1 + x_1 x_2^2)^2 + (2.625 - x_1 + x_1 x_2^3)^2
    return (1.5 - x + x*y)**2 + (2.25 - x + x*y**2)**2 + (2.625 - x + x*y**3)**2

def gradient_beale(x, y):
    return np.array([2*(1.5 - x + x*y)*(y-1) + 2*(2.25 - x + x*y**2)*(y**2-1) + 2*(2.625 - x + x*y**3)*(y**3-1),
                     2*(1.5 - x + x*y)*x + 2*(2.25 - x + x*y**2)*x*y*2 + 2*(2.625 - x + x*y**3)*x*y**2*3])
    
